In [ ]:
# Import a bunch of shit you don't need:

In [1]:
import os, sys, shutil, bz2, random, resource, warnings, subprocess, copy, re, glob, json, collections, gc, contextlib
from pathlib import Path
import json_stream
import xml.etree.ElementTree as ET
from subprocess import Popen, PIPE, STDOUT
import pandas as pd
pd.set_option('display.max_columns', 50)
import numpy as np
from Bio import SeqIO, bgzf
import Bio.Data.CodonTable
from Bio.SeqIO.QualityIO import FastqGeneralIterator
from Bio import Seq, SeqIO, SearchIO, SeqRecord
from mpire import WorkerPool
import jellyfish
from Bio import Align

In [ ]:
# Locate a bunch of UMI trimmed reads:

In [2]:
%%bash
pwd

/home/sulab/tRNA-charge-seq/utils


In [3]:
%%bash
ls -lrth ../projects/tRNAseq_third-gen/data/UMI_trimmed/*-trimmed.fastq.bz2

-rw-r--r-- 1 sulab sulab  14M Jan  5 20:23 ../projects/tRNAseq_third-gen/data/UMI_trimmed/85p3_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  18M Jan  5 20:24 ../projects/tRNAseq_third-gen/data/UMI_trimmed/40p3_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  36M Jan  5 20:24 ../projects/tRNAseq_third-gen/data/UMI_trimmed/85p2_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  38M Jan  5 20:25 ../projects/tRNAseq_third-gen/data/UMI_trimmed/100p3_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  41M Jan  5 20:25 ../projects/tRNAseq_third-gen/data/UMI_trimmed/100p4_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  47M Jan  5 20:25 ../projects/tRNAseq_third-gen/data/UMI_trimmed/70p1_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  62M Jan  5 20:26 ../projects/tRNAseq_third-gen/data/UMI_trimmed/100p2_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  61M Jan  5 20:27 ../projects/tRNAseq_third-gen/data/UMI_trimmed/100p1_UMI-trimmed.fastq.bz2
-rw-r--r-- 1 sulab sulab  68M Jan  5 20:27 ../projects/tRNAs

In [ ]:
# Loop through and count them all.
### BE CAREFUL ###
# This is obviously a very memory hungry operation,
# so don't do this without being prepared for this.
# There are more memory efficient approximate ways of doing this
# e.g. Kmer based clustering, but this is fast and works on
# our workstation with 32 Gb memory.

In [ ]:
UMI_folders = '/home/sulab/tRNA-charge-seq/projects/tRNAseq_third-gen/data/UMI_trimmed/*-trimmed.fastq.bz2'

In [7]:
tot = 0
seq_obs = dict()
for fnam in glob.glob(UMI_folders):
    with bz2.open(fnam, 'rt') as fh_in:
        print(fnam.split('/')[-1])
        for title, seq, qual in FastqGeneralIterator(fh_in):
            tot += 1
            try:
                seq_obs[seq] += 1
            except:
                seq_obs[seq] = 1
        print(tot)
        print(len(seq_obs))


0m_3_UMI-trimmed.fastq.bz2
3623540
935857
16h_4_UMI-trimmed.fastq.bz2
8139268
1937689
70p1_UMI-trimmed.fastq.bz2
11543739
2585326
8m_2_UMI-trimmed.fastq.bz2
15181672
3168403
10p4_UMI-trimmed.fastq.bz2
18673052
3793616
R5_UMI-trimmed.fastq.bz2
23610645
4471869
40h_2_UMI-trimmed.fastq.bz2
27572261
5031728
8m_3_UMI-trimmed.fastq.bz2
31896531
5655535
4h_1_UMI-trimmed.fastq.bz2
35586694
6137085
0p2_UMI-trimmed.fastq.bz2
39528489
6721843
0m_1_UMI-trimmed.fastq.bz2
44754382
7378781
L8_UMI-trimmed.fastq.bz2
49165508
7857122
32m_2_UMI-trimmed.fastq.bz2
53662481
8385378
40h_NoOx_1_UMI-trimmed.fastq.bz2
59293759
8928283
R3_UMI-trimmed.fastq.bz2
63480280
9352403
40h_4_UMI-trimmed.fastq.bz2
66647665
9759725
1h_1_UMI-trimmed.fastq.bz2
70314250
10168878
L6_UMI-trimmed.fastq.bz2
74415743
10586610
1h_2_UMI-trimmed.fastq.bz2
75559649
10700123
55p3_UMI-trimmed.fastq.bz2
79295085
11155231
55p1_UMI-trimmed.fastq.bz2
84496844
11794464
10p2_UMI-trimmed.fastq.bz2
87005100
12092834
16m_1_UMI-trimmed.fastq.bz2


In [ ]:
# ^^ More than 10:1 duplication rate. That is huge!

In [ ]:
# Iteratively remove sequences with the least number of observations.
# Don't try deleting from the dict to save memory. It will be too slow.
# Reassignment is pretty quick though.
# Max sequences after prunning should be 1 million.

In [47]:
max_seqs = 1e6
min_obs = 2
Nseqs = len(seq_obs)
while Nseqs > max_seqs:
    print('Getting rid of sequences with less than {} observations'.format(min_obs))
    print('{} sequences in the stack before pruning.'.format(Nseqs))
    new_dict = {k: v for k, v in seq_obs.items() if v >= min_obs}
    min_obs += 1
    Nseqs = len(new_dict)
    print('{} sequences in the stack after pruning.\n'.format(Nseqs))

Getting rid of sequences with less than 2 observations
17238238 sequences in the stack before pruning.
17143129 sequences in the stack after pruning.

Getting rid of sequences with less than 3 observations
17143129 sequences in the stack before pruning.
9247158 sequences in the stack after pruning.

Getting rid of sequences with less than 4 observations
9247158 sequences in the stack before pruning.
6150343 sequences in the stack after pruning.

Getting rid of sequences with less than 5 observations
6150343 sequences in the stack before pruning.
4681645 sequences in the stack after pruning.

Getting rid of sequences with less than 6 observations
4681645 sequences in the stack before pruning.
3843673 sequences in the stack after pruning.

Getting rid of sequences with less than 7 observations
3843673 sequences in the stack before pruning.
3294359 sequences in the stack after pruning.

Getting rid of sequences with less than 8 observations
3294359 sequences in the stack before pruning.
2

In [ ]:
# ^^ Nice!!!
# So we can approx. cut away this many of the alignments (%):

In [56]:
sum(v for v in new_dict.values()) / 437410038 * 100

78.77542787438271

In [ ]:
# So that should cut the alignment run time down to 1/4 of the original

In [ ]:
# Time to sort them:

In [53]:
seq_obs_sorted = sorted(new_dict.items(), key=lambda x: x[1], reverse=True)

In [54]:
seq_obs_sorted[0]

('ATCTGAGGGTCCAGGGTTCATGTCCCTGTTCGGGCGCCA', 1665731)

In [ ]:
# ^^ So the most frequent read is seen a crazy 1.6 million times!
# Apparently, a Lys-TTT read...
# No need to run the same alignment 1.6 million times that is for sure.

In [ ]:
# Write "common sequences" to a compressed file for future use:

In [60]:
fnam_out = '/home/sulab/tRNA-charge-seq/utils/common-seqs.fasta.bz2'
with bz2.open(fnam_out, 'wt') as fh_out:
    for seq_idx, seq_tup in enumerate(seq_obs_sorted):
        fh_out.write('>{}\n{}\n'.format(seq_idx, seq_tup[0]))